In [ ]:
!pip install mlflow
!pip install pyngrok


import mlflow
import mlflow.sklearn

mlflow.set_experiment("Exp 5- ML Algos with HP Tuning")

from google.colab import userdata
ngRoktoken=userdata.get('ngroktoen')

get_ipython().system_raw('mlflow ui --port 2000 &')
mlflow.set_tracking_uri('http://localhost:2000')
from pyngrok import ngrok
ngrok.set_auth_token(ngRoktoken)

public_url=ngrok.connect(2000).public_url
print('mlflow ui url: ', public_url)

In [ ]:
!pip install optuna


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 12.4 MB/s eta 0:00:00


In [ ]:
df=pd.read_csv('reddit_preprocessing.csv').dropna()
df.shape

(36662, 2)

In [ ]:
# =================== Imports ===================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
import mlflow.sklearn
import optuna

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE
from mlflow.models.signature import infer_signature
# ==============================================

# Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# Step 2: Remove rows with NaN category values (due to mapping)
df = df.dropna(subset=['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # trigram
max_features = 1000   # set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled
)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # log the model with input_example and signature
        input_example = X_test[:1]
        pred_example = model.predict(input_example)
        signature = infer_signature(input_example, pred_example)

        mlflow.sklearn.log_model(
            sk_model=model,
            artifact_path=f"{model_name}_model",
            input_example=input_example,
            signature=signature
        )

# Step 6: Optuna objective function for KNN
def objective_knn(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 3, 15)

    model = KNeighborsClassifier(
        n_neighbors=n_neighbors
    )

    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))

# Step 7: Run Optuna study and log best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_knn, n_trials=30)

    # get the best parameters and log only the best model
    best_params = study.best_params
    best_model = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'])

    # log the best model with mlflow, passing the algo_name as 'KNN'
    log_mlflow("KNN", best_model, X_train, X_test, y_train, y_test)

# Run the full KNN tuning and logging experiment
run_optuna_experiment()
